# Nifti Read Example

The purpose of this notebook is to illustrate reading Nifti files and iterating over patches of the volumes loaded from them.

In [1]:
%matplotlib inline

import os
import sys
from glob import glob
import tempfile
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import torch
from torch.utils.data import DataLoader

import monai
from monai.data import ArrayDataset, GridPatchDataset, create_test_image_3d
from monai.transforms import (Compose, LoadNifti, AddChannel, Transpose,
                              ScaleIntensity, ToTensor, RandSpatialCrop)

monai.config.print_config()

MONAI version: 0.1.0+58.g9df3c78.dirty
Python version: 3.6.9 |Anaconda, Inc.| (default, Jul 30 2019, 19:07:31)  [GCC 7.3.0]
Numpy version: 1.17.4
Pytorch version: 1.5.0
Ignite version: 0.3.0


Create a number of test Nifti files:

In [2]:
tempdir = tempfile.mkdtemp()

for i in range(5):
    im, seg = create_test_image_3d(128, 128, 128)
    
    n = nib.Nifti1Image(im, np.eye(4))
    nib.save(n, os.path.join(tempdir, 'im%i.nii.gz'%i))
    
    n = nib.Nifti1Image(seg, np.eye(4))
    nib.save(n, os.path.join(tempdir, 'seg%i.nii.gz'%i))

Create a data loader which yields uniform random patches from loaded Nifti files:

In [3]:
images = sorted(glob(os.path.join(tempdir, 'im*.nii.gz')))
segs = sorted(glob(os.path.join(tempdir, 'seg*.nii.gz')))

imtrans = Compose([
    LoadNifti(image_only=True),
    ScaleIntensity(),
    AddChannel(),
    RandSpatialCrop((64, 64, 64), random_size=False),
    ToTensor()
])    

segtrans = Compose([
    LoadNifti(image_only=True),
    AddChannel(),
    RandSpatialCrop((64, 64, 64), random_size=False),
    ToTensor()
])    
    
ds = ArrayDataset(images, imtrans, segs, segtrans)

loader = DataLoader(ds, batch_size=10, num_workers=2, pin_memory=torch.cuda.is_available())
im, seg = monai.utils.misc.first(loader)
print(im.shape, seg.shape)

torch.Size([5, 1, 64, 64, 64]) torch.Size([5, 1, 64, 64, 64])


Alternatively create a data loader which yields patches in regular grid order from loaded images:

In [4]:
imtrans = Compose([
    LoadNifti(image_only=True),
    ScaleIntensity(),
    AddChannel(),
    ToTensor()
])    

segtrans = Compose([
    LoadNifti(image_only=True),
    AddChannel(),
    ToTensor()
])    
    
ds = ArrayDataset(images, imtrans, segs, segtrans)
ds = GridPatchDataset(ds, (64, 64, 64))

loader = DataLoader(ds, batch_size=10, num_workers=2, pin_memory=torch.cuda.is_available())
im, seg = monai.utils.misc.first(loader)
print(im.shape, seg.shape)

torch.Size([10, 1, 64, 64, 64]) torch.Size([10, 1, 64, 64, 64])


In [5]:
!rm -rf {tempdir}